# ETL Pipeline – Predictive Maintenance BI

## Overview
This notebook implements the **Extract, Transform, Load (ETL)** pipeline for the Predictive Maintenance Analytics project. It transforms raw sensor data from CNC milling machines into a **star schema** data model suitable for Business Intelligence analysis.

### Pipeline Steps:
1. **Extract**: Load raw data from CSV
2. **Transform**: Clean, rename columns, create dimensions and fact table
3. **Load**: Export processed tables to CSV for Power BI consumption

### Data Model (Star Schema):
- **Fact Table**: `Fact_MachineEvents` – Machine sensor readings and failure indicators
- **Dim_Date**: Calendar dimension with date attributes
- **Dim_Machine**: Machine information
- **Dim_ProductType**: Product type classifications (L, M, H)

---

## Step 1: Extract – Load Raw Data
Load the AI4I 2020 Predictive Maintenance dataset containing 10,000 machine event records.

In [1]:
import pandas as pd

# Path to raw dataset
raw_path = "C:/Users/R I B/Documents/Predictive-Maintenance-Analytics-for-Smart-Manufacturing/data/raw/predictive_maintenance.csv"  

df_raw = pd.read_csv(raw_path)
df_raw.head()


,Type,Air temperature,Process temperature,Rotational speed,Torque,Tool wear,Machine failure,TWF,HDF,PWF,OSF,RNF
0,M,298.1,308.6,1551,42.8,0,0,0,0,0,0,0
1,L,298.2,308.7,1408,46.3,3,0,0,0,0,0,0
2,L,298.1,308.5,1498,49.4,5,0,0,0,0,0,0
3,L,298.2,308.6,1433,39.5,7,0,0,0,0,0,0
4,L,298.2,308.7,1408,40.0,9,0,0,0,0,0,0


## Step 2: Transform – Data Cleaning & Column Renaming
- Rename columns to BI-friendly names (no spaces, CamelCase)
- Ensure failure indicator columns are integers
- Check for missing values

In [2]:
# Rename columns to BI‑friendly names
df = df_raw.rename(columns={
    "Type": "ProductType",
    "Air temperature": "AirTemperature",
    "Process temperature": "ProcessTemperature",
    "Rotational speed": "RotationalSpeed",
    "Torque": "Torque",
    "Tool wear": "ToolWear",
    "Machine failure": "MachineFailure"
})

# Ensure failure columns are integers
for col in ["MachineFailure", "TWF", "HDF", "PWF", "OSF", "RNF"]:
    df[col] = df[col].astype(int)

df.isna().sum()
df.head()


,ProductType,AirTemperature,ProcessTemperature,RotationalSpeed,Torque,ToolWear,MachineFailure,TWF,HDF,PWF,OSF,RNF
0,M,298.1,308.6,1551,42.8,0,0,0,0,0,0,0
1,L,298.2,308.7,1408,46.3,3,0,0,0,0,0,0
2,L,298.1,308.5,1498,49.4,5,0,0,0,0,0,0
3,L,298.2,308.6,1433,39.5,7,0,0,0,0,0,0
4,L,298.2,308.7,1408,40.0,9,0,0,0,0,0,0


## Step 3: Transform – Feature Engineering
- Convert temperatures from Kelvin to Celsius
- Generate synthetic timestamps for time-series analysis
- Assign machines (simulate 3 machines in rotation)

In [3]:
# Convert temperatures to Celsius (dataset already looks near 300K, but we keep Kelvin as is)
# If they are already in Kelvin around 300, we can still compute Celsius:
df["AirTempC"] = df["AirTemperature"] - 273.15
df["ProcessTempC"] = df["ProcessTemperature"] - 273.15

# Sort by index and create synthetic timestamp
df = df.reset_index(drop=True)
df["Timestamp"] = pd.to_datetime("2025-01-01") + pd.to_timedelta(df.index, unit="min")
df["Date"] = df["Timestamp"].dt.date
df["DateID"] = df["Timestamp"].dt.strftime("%Y%m%d").astype(int)

# Simulate 3 machines
df["MachineID"] = (df.index % 3) + 1

df.head()


,ProductType,AirTemperature,ProcessTemperature,RotationalSpeed,Torque,ToolWear,MachineFailure,TWF,HDF,PWF,OSF,RNF,AirTempC,ProcessTempC,Timestamp,Date,DateID,MachineID
0,M,298.1,308.6,1551,42.8,0,0,0,0,0,0,0,24.95,35.45,2025-01-01 00:00:00,2025-01-01,20250101,1
1,L,298.2,308.7,1408,46.3,3,0,0,0,0,0,0,25.05,35.55,2025-01-01 00:01:00,2025-01-01,20250101,2
2,L,298.1,308.5,1498,49.4,5,0,0,0,0,0,0,24.95,35.35,2025-01-01 00:02:00,2025-01-01,20250101,3
3,L,298.2,308.6,1433,39.5,7,0,0,0,0,0,0,25.05,35.45,2025-01-01 00:03:00,2025-01-01,20250101,1
4,L,298.2,308.7,1408,40.0,9,0,0,0,0,0,0,25.05,35.55,2025-01-01 00:04:00,2025-01-01,20250101,2


## Step 4: Create Dimension Tables
Build the star schema dimensions:
- **Dim_ProductType**: Product type lookup (L, M, H)
- **Dim_Date**: Calendar dimension with day, month, quarter, year
- **Dim_Machine**: Machine information with names, lines, and locations

In [4]:
# Dim_ProductType
dim_product = (
    df[["ProductType"]]
    .drop_duplicates()
    .reset_index(drop=True)
)
dim_product["ProductTypeID"] = dim_product.index + 1
dim_product = dim_product[["ProductTypeID", "ProductType"]]

# Attach ProductTypeID to main df
df = df.merge(dim_product, on="ProductType", how="left")

# Dim_Date
dim_date = (
    df[["Date", "DateID"]]
    .drop_duplicates()
    .sort_values("Date")
)
dim_date["Day"] = pd.to_datetime(dim_date["Date"]).dt.day
dim_date["Month"] = pd.to_datetime(dim_date["Date"]).dt.month
dim_date["Quarter"] = pd.to_datetime(dim_date["Date"]).dt.quarter
dim_date["Year"] = pd.to_datetime(dim_date["Date"]).dt.year
dim_date["Shift"] = "Day"  # simple placeholder
dim_date = dim_date[["DateID", "Date", "Day", "Month", "Quarter", "Year", "Shift"]]

# Dim_Machine
import pandas as pd  # in case this cell runs alone

dim_machine = pd.DataFrame({
    "MachineID": [1, 2, 3],
    "MachineName": ["Machine A", "Machine B", "Machine C"],
    "Line": ["Line 1", "Line 1", "Line 2"],
    "Location": ["Plant 1", "Plant 1", "Plant 1"]
})

dim_product, dim_date.head(), dim_machine


(   ProductTypeID ProductType
 0              1           M
 1              2           L
 2              3           H,
         DateID        Date  Day  Month  Quarter  Year Shift
 0     20250101  2025-01-01    1      1        1  2025   Day
 1440  20250102  2025-01-02    2      1        1  2025   Day
 2880  20250103  2025-01-03    3      1        1  2025   Day
 4320  20250104  2025-01-04    4      1        1  2025   Day
 5760  20250105  2025-01-05    5      1        1  2025   Day,
    MachineID MachineName    Line Location
 0          1   Machine A  Line 1  Plant 1
 1          2   Machine B  Line 1  Plant 1
 2          3   Machine C  Line 2  Plant 1)

## Step 5: Create Fact Table & Load
Build the fact table `Fact_MachineEvents` with:
- Surrogate key (EventID)
- Foreign keys to all dimensions
- Measures: Temperature, Speed, Torque, Tool Wear
- Failure indicators: MachineFailure, TWF, HDF, PWF, OSF, RNF

Export all tables to CSV files in the `data/processed/` folder.

In [5]:
# Build fact table

fact = df[[
    "DateID",
    "MachineID",
    "ProductTypeID",
    "AirTempC",
    "ProcessTempC",
    "RotationalSpeed",
    "Torque",
    "ToolWear",
    "MachineFailure",
    "TWF", "HDF", "PWF", "OSF", "RNF"
]].copy()

# Create EventID from the row index
fact = fact.reset_index(drop=True)
fact["EventID"] = fact.index + 1

fact = fact[[
    "EventID",
    "DateID",
    "MachineID",
    "ProductTypeID",
    "AirTempC",
    "ProcessTempC",
    "RotationalSpeed",
    "Torque",
    "ToolWear",
    "MachineFailure",
    "TWF", "HDF", "PWF", "OSF", "RNF"
]]

# Save processed tables (note ../ because notebook is in etl/)
fact.to_csv("../data/processed/Fact_MachineEvents.csv", index=False)
dim_product.to_csv("../data/processed/Dim_ProductType.csv", index=False)
dim_date.to_csv("../data/processed/Dim_Date.csv", index=False)
dim_machine.to_csv("../data/processed/Dim_Machine.csv", index=False)


## ETL Pipeline Complete ✅

### Output Files:
| File | Description | Records |
|------|-------------|---------|
| `Fact_MachineEvents.csv` | Machine sensor readings fact table | 10,000 |
| `Dim_Date.csv` | Date dimension | 7 |
| `Dim_Machine.csv` | Machine dimension | 3 |
| `Dim_ProductType.csv` | Product type dimension | 3 |

### Next Steps:
1. Load CSV files into Power BI
2. Create relationships in data model
3. Build DAX measures and visualizations

In [6]:
# Data Quality Summary
print("=" * 50)
print("ETL PIPELINE - DATA QUALITY REPORT")
print("=" * 50)

print("\n📊 RECORD COUNTS:")
print(f"   Raw records loaded:        {len(df_raw):,}")
print(f"   Fact table records:        {len(fact):,}")
print(f"   Dim_Date records:          {len(dim_date):,}")
print(f"   Dim_Machine records:       {len(dim_machine):,}")
print(f"   Dim_ProductType records:   {len(dim_product):,}")

print("\n✅ DATA QUALITY CHECKS:")
print(f"   Missing values in fact:    {fact.isna().sum().sum()}")
print(f"   Duplicate EventIDs:        {fact['EventID'].duplicated().sum()}")

print("\n📈 KEY STATISTICS:")
print(f"   Failure rate:              {(fact['MachineFailure'].sum() / len(fact) * 100):.2f}%")
print(f"   Total failures:            {fact['MachineFailure'].sum():,}")
print(f"   Avg Air Temperature:       {fact['AirTempC'].mean():.2f}°C")
print(f"   Avg Tool Wear:             {fact['ToolWear'].mean():.1f} min")

print("\n" + "=" * 50)
print("ETL COMPLETE - Files saved to data/processed/")
print("=" * 50)

ETL PIPELINE - DATA QUALITY REPORT

📊 RECORD COUNTS:
   Raw records loaded:        10,000
   Fact table records:        10,000
   Dim_Date records:          7
   Dim_Machine records:       3
   Dim_ProductType records:   3

✅ DATA QUALITY CHECKS:
   Missing values in fact:    0
   Duplicate EventIDs:        0

📈 KEY STATISTICS:
   Failure rate:              3.39%
   Total failures:            339
   Avg Air Temperature:       26.85°C
   Avg Tool Wear:             108.0 min

ETL COMPLETE - Files saved to data/processed/
